In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Feb 22 10:40:56 2020

@author: marcelo

ar1:
- array (nx,2), onde:
    nx: n'umero total de picos em todas as escalas
    x: 'indice do pico
    ar1[x,0]: canal inteiro do pico
    ar1[x,1]: valor da escala
    
xpk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise
    x: canal inteiro do pico
    
ypk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise (= nx(xpk))
    x: valor da escala

"""

import numpy as np

from spectrum_analysis import (cwt_complete_analysis, gaussian_complete_analysis, cwt_calc_peaks)
from teste_datas import (Spec)

from machine_selection import set_screen_size_by_machine
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.layouts import gridplot

from bokeh import palettes

In [2]:
# 2019-03-15
# https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html
import pandas as pd
import csv

In [3]:
# fn2 = '../Efluentes-gasosos/Filtros-espectros/2018/CTP/CTP2203-3.Chn'
fn2 = '../Pesquisa/2020_Mariana_PR/2020_Mariana_PR_Espectros/AgAm3-2911.chn'

In [4]:
spec2 = Spec()
if spec2.readSp( fn2 ) >= 0:
    print (spec2.spLngData)
    
counts = spec2.spCounts
print(len(counts))

ret = cwt_complete_analysis(counts)
# ret = gaussian_complete_analysis(counts)

(xs, y0s, ar1, lblimag1, num_feat_imag1, ridges, xpks, ypks,
 ind_ypks, retang_indicess, peaks_masks, cwtpks, nscpks) = ret

4096
4096


In [5]:
# 2020-05-15 Espectros Mariana PR.

In [6]:
np.float32(4.4)

4.4

In [7]:
# Desativado:

In [8]:
(nscpk_ma, cwtpk_ma, pospk_ma) = peaks_masks
valid_peaks = ~(cwtpk_ma.mask | nscpk_ma.mask | ~pospk_ma)

In [9]:
default_plot_width, default_plot_height = set_screen_size_by_machine()

# Bokeh graph w/ peaks in all scales

# Gráfico de ridges com qualidade dos picos pelas cores e tamanhos:

# Navy 2:     pontos dos ridgest
# Vermelho 8: picos válidos
# Laranja 8:  inválido, aparece em poucas escalas  
# Verde 6:    inválido, muito pequeno (pelo coeficiente cwt)
# Ciano 3:    inválido, forma ruim (cwt perto dos extremos do ridge)

bokridges1 = figure(
   plot_width =default_plot_width,
   plot_height=int(default_plot_height*0.5),
   tools="pan,box_zoom,reset,save",
   # x_axis_label='channels',
   y_axis_label='scale',
   title=fn2,
)
bokridges1.circle(ar1[:,0], ar1[:,1], color="navy",size=2)
bokridges1.circle(xpks[valid_peaks], ypks[valid_peaks], color="red",size=8)
bokridges1.circle(xpks[nscpk_ma.mask], ypks[nscpk_ma.mask], color="orange",size=8)
bokridges1.circle(xpks[cwtpk_ma.mask], ypks[cwtpk_ma.mask], color="green",size=6)
bokridges1.circle(xpks[~pospk_ma], ypks[~pospk_ma], color="cyan",size=3)
bokridges2 = figure(
   plot_width =default_plot_width,
   plot_height=int(default_plot_height*0.5),
   tools="pan,box_zoom,reset,save",
   y_axis_type="log", y_range=[1.0e1, 1.0e5],
   x_axis_label='channels', y_axis_label='counts'
)
bokridges2.line(xs, y0s, color="green")
# https://docs.bokeh.org/en/latest/docs/reference/layouts.html
p = gridplot([bokridges1, bokridges2], ncols=1)
show(p)

In [10]:
valid_xpks = np.asarray(xpks)[valid_peaks]
valid_xpks

array([ 111,  153,  226,  345,  350,  460,  492,  593,  692,  845,  878,
       1158, 1279, 1426, 1461, 1495, 1525, 1823, 2013, 2286, 2512, 2812,
       3669, 1744,  322])

In [11]:
valid_ypks = np.asarray(ypks)[valid_peaks]
valid_ypks

array([ 2.62987755,  2.41991837,  4.09959184,  2.83983673, 11.86808163,
        3.67967347,  3.46971429,  3.46971429,  3.46971429,  6.82906122,
        2.62987755,  6.19918367,  6.61910204,  9.13861224,  4.30955102,
        4.30955102,  3.67967347,  3.88963265,  3.67967347,  5.35934694,
        9.7684898 ,  4.09959184,  5.35934694, 10.18840816,  2.83983673])

In [12]:
valid_cwtpks = np.asarray(cwtpks)[valid_peaks]
valid_cwtpks

array([ 315.60988264,  384.37031546,  942.31011778,   97.92719052,
        145.50552394,  587.82621701,  164.97722366,  289.10352168,
         49.01765747,   57.01773419,   79.83384172,   30.00778554,
       1465.24540443,   53.05132051,   95.2971835 ,   44.6034454 ,
         67.93598   ,   33.48617446,   75.17344877,   60.92771669,
         43.74211686,   32.52200065,  313.236379  ,   46.02902073,
         56.80832437])

In [13]:
xs

array([0.000e+00, 1.000e+00, 2.000e+00, ..., 4.093e+03, 4.094e+03,
       4.095e+03])

In [14]:
y0s

array([   0,    0,    0, ..., 2653, 2587, 2602])

In [15]:
step2 = cwt_calc_peaks(valid_xpks, valid_ypks, valid_cwtpks, xs, y0s)

In [16]:
xpks[pospk_ma]


array([ 111,  153,  226,  345,  350,  460,  492,  593,  692,  845,  868,
        878, 1040, 1062, 1158, 1177, 1279, 1426, 1461, 1495, 1525, 1625,
       1648, 1680, 1823, 1910, 2013, 2040, 2099, 2286, 2318, 2413, 2465,
       2512, 2569, 2580, 2614, 2669, 2683, 2736, 2798, 2812, 2831, 2842,
       2853, 2877, 2943, 2989, 3005, 3108, 3126, 3241, 3343, 3413, 3535,
       3557, 3570, 3669, 3774, 3793, 3822, 3849, 1012, 1744, 2549,  322,
       1052, 1776, 3222, 1896, 2628,  792, 2177, 2652, 3025, 3044, 3495,
       1921, 1093, 1859])

In [17]:
(azes, bezes, centrpk, ypkregs, fwhms, gau_areas, peak_constructed_spectrum,
           baseline_spectrum, bl_median_smoother) = step2

In [18]:
azes

[array([106., 107., 108., 109., 110., 111., 112., 113., 114., 115., 116.]),
 array([148., 149., 150., 151., 152., 153., 154., 155., 156., 157., 158.]),
 array([221., 222., 223., 224., 225., 226., 227., 228., 229., 230., 231.]),
 array([340., 341., 342., 343., 344., 345., 346., 347., 348., 349., 350.]),
 array([345., 346., 347., 348., 349., 350., 351., 352., 353., 354., 355.]),
 array([455., 456., 457., 458., 459., 460., 461., 462., 463., 464., 465.]),
 array([487., 488., 489., 490., 491., 492., 493., 494., 495., 496., 497.]),
 array([588., 589., 590., 591., 592., 593., 594., 595., 596., 597., 598.]),
 array([687., 688., 689., 690., 691., 692., 693., 694., 695., 696., 697.]),
 array([840., 841., 842., 843., 844., 845., 846., 847., 848., 849., 850.]),
 array([873., 874., 875., 876., 877., 878., 879., 880., 881., 882., 883.]),
 array([1153., 1154., 1155., 1156., 1157., 1158., 1159., 1160., 1161.,
        1162., 1163.]),
 array([1274., 1275., 1276., 1277., 1278., 1279., 1280., 1281., 1282.

In [19]:
len(azes)

25

In [20]:
bezes

[array([  0. ,  49.9,  58.8,  92.7, 210.6, 273.5,  96.4,  21.3, -29.8,
        -43.9,   0. ]),
 array([  0.,   1.,  59., 137., 279., 284., 197.,  41., -37., -15.,   0.]),
 array([  0. ,  15.5,  73. , 192.5, 437. , 668.5, 715. , 429.5, 157. ,
         96.5,   0. ]),
 array([  0.,  24.,  54.,   4., 102., 114.,  63.,  53.,  28.,  79.,   0.]),
 array([   0. ,  -55.5,  -70. ,  -99.5,  -53. , -136.5,  -89. , -111.5,
         -78. ,  -42.5,    0. ]),
 array([  0.,  14.,  18., 152., 305., 423., 439., 159.,  64.,  35.,   0.]),
 array([  0. , -24.8,  22.4,  22.6, 109.8, 102. ,  99.2,  57.4,  16.6,
         28.8,   0. ]),
 array([  0. ,  13.5,  44. , 141.5, 177. , 207.5, 201. ,  65.5,  49. ,
         31.5,   0. ]),
 array([ 0. , 34.7, 42.4, 54.1, 13.8, 50.5, 56.2, 28.9, 28.6,  8.3,  0. ]),
 array([  0. ,  -8.4,  14.2,  21.8,   9.4,  -5. ,  -8.4,  27.2,   1.8,
        -18.6,   0. ]),
 array([  0. ,   1.3,  18.6,  22.9,  53.2,  57.5,  26.8,   4.1, -26.6,
         -1.3,   0. ]),
 array([  0. , -21.7

In [21]:
centrpk

[109.76901987662785,
 152.33826638477802,
 226.4769258394685,
 345.38579654510556,
 350.043507817811,
 460.2299564947172,
 492.58986175115206,
 592.9559376679205,
 691.5118110236219,
 842.5,
 876.6581469648562,
 1154.9787234042553,
 1278.6965874171115,
 1426.8217821782177,
 1460.1984732824428,
 1494.469696969697,
 1525.4715127701375,
 1821.8587570621469,
 2012.5247524752476,
 2285.762039660057,
 2511.963746223565,
 2811.95020746888,
 3668.6298633017873,
 1743.9590163934429,
 322.22937062937064]

In [22]:
#azes, bezes, centrpk, ypkregs, fwhms, gau_areas, peak_constructed_spectrum,
#           baseline_spectrum, bl_median_smoother) = step2

In [23]:
bezes[0]

array([  0. ,  49.9,  58.8,  92.7, 210.6, 273.5,  96.4,  21.3, -29.8,
       -43.9,   0. ])

In [24]:
gau_areas

[775.7259619502651,
 906.2336141267638,
 2891.7059795068794,
 250.1151221409895,
 759.7307634959105,
 1709.0058973732223,
 465.7585293330288,
 816.1880052324423,
 138.38511492661289,
 225.82937213888323,
 196.22067328345963,
 113.23773352692267,
 5713.468995795589,
 243.06716329804183,
 299.8375686451004,
 140.33771125333934,
 197.51244008032197,
 100.09440404551438,
 218.55416371531211,
 213.77691617812425,
 207.20654964869436,
 99.80160667164243,
 1099.051643157564,
 222.67674883759994,
 145.09372640632992]

In [25]:
# 2020-02-24
# Tudo em progresso! Agora:
# - Fazer os gr'aficos
# - fazer em v'arias passadas o c'alculo at'e n~ao haver picos na linha-base.

In [26]:
# Bokeh graph w/ peaks in all scales
bokpksallscl = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   y_axis_type="log",
   y_range=[10**1, 10**5],
   title=fn2,
   # y_range=[-1000.0, 1000.0],
   x_axis_label='channels', y_axis_label='counts'
)
# bokpksallscl.circle(ar1[:,0], ar1[:,1], legend_label="cwt peaks", color="navy",size=2)
bokpksallscl.line(xs, y0s, color="green")
bokpksallscl.line(xs, peak_constructed_spectrum, color="red")
bokpksallscl.line(xs, bl_median_smoother, color="orange")
bokpksallscl.line(xs, peak_constructed_spectrum+bl_median_smoother, color="blue")

# bokpksallscl.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
show(bokpksallscl)

In [27]:
# Bokeh graph w/ peaks in all scales
bok2 = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   # y_axis_type="log",
   # y_range=[10**1, 10**5],
   title=fn2,
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
# bokpksallscl.circle(ar1[:,0], ar1[:,1], legend_label="cwt peaks", color="navy",size=2)
# bokpksallscl.line(xs, y0s, color="green")
bok2.line(xs, peak_constructed_spectrum, color="red")
# bokpksallscl.line(xs, bl_median_smoother, color="cyan")
# bokpksallscl.line(xs, peak_constructed_spectrum+bl_median_smoother, color="blue")

# bokpksallscl.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
show(bok2)

In [28]:
# 2020-02-25 Segunda passada:
# 2020-05-15 Por enquanto deixarei desativada; mas posso fazer manualmente, bastando rodar
#            as duas células seguintes e repetir o trecho que se inicia logo após
#            a determinação de valid_peaks.

In [29]:
# 2020-05-17 Continuando para cada espectro. Este é AgAm3-2911.chn

In [30]:
spec2.diahora

datetime.datetime(2020, 2, 3, 14, 36, 28)

In [31]:
spec2.spType

-1

In [32]:
print(spec2.enCalZinpt)
print(spec2.enCalSlope)
print(spec2.enCalQuadr)
print(spec2.pkShpZinpt)
print(spec2.pkShpSlope)
print(spec2.pkShpQuadr)
print(spec2.lenDetDesc)
print(spec2.detDesc)
print(spec2.lenSamDesc)
print(spec2.samDesc)

2.6155457496643066
0.3980768024921417
-1.0149523888003387e-07
2.674441337585449
-0.0004642767598852515
2.759846893241047e-07
13
MCB 1 Input 1
59
Agua Am 3- 29/11 - m=101.4g - Sel 17/12/2019 - F100 (Det C)


In [49]:
print(spec2.spLVTime)

250000.0


In [50]:
print(spec2.spRLTime)

250119.88


In [51]:
print(spec2.diahora)

2020-02-03 14:36:28
